# Note on packages

alembic is for migrations
ipykernel is for using sqlalchemy easier with Jupyter (not sure if/why needed)

In [ ]:
!poetry add sqlalchemy ipykernel alembic bibtexparser pandas

# Create db and all missing tables

In [5]:
import sys
from pathlib import Path

# Add the root and models directories to the Python path
sys.path.append(str(Path('.').resolve()))
sys.path.append(str((Path('.') / 'models').resolve()))

# Now you can import
from models.db_models import Base
from config import NotebookConfig
from sqlalchemy import create_engine

# Create the database
engine = create_engine(f"sqlite:///{NotebookConfig.DB_FILE}")
Base.metadata.create_all(engine)

print(f"✅ Database created at: {NotebookConfig.DB_FILE}")

✅ Database created at: cybercrime_analysis.db


# Import papers (sources) that Umind found

In [ ]:
import sys
from pathlib import Path
import csv
from datetime import datetime

# Configuration section.
TOPIC_NAME = "Cybercrime and intervention metrics"

# Add the root and models directories to the Python path.
sys.path.append(str(Path('.').resolve()))
sys.path.append(str((Path('.') / 'models').resolve()))

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from config import NotebookConfig
from db_models import Source, generate_cite_key, UnmindSourceRelevanceRating  # Import from models

def unmind_import():
    csv_path = NotebookConfig.INPUT_DIR / "undermind-cybercrime-table-export.csv"
    if not csv_path.exists():
        print(f"CSV file not found: {csv_path}")
        return

    engine = create_engine(f"sqlite:///{NotebookConfig.DB_FILE}")
    Session = sessionmaker(bind=engine)
    session = Session()

    sources_to_add = []
    ratings_to_add = []

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Get CSV fields; adjust keys if necessary.
            title = row.get("Title", "").strip()
            authors = row.get("Authors", "").strip()
            try:
                year = int(row.get("Year", "").strip())
            except (ValueError, AttributeError):
                year = None

            date_str = row.get("Date", "").strip()
            date_obj = None
            if date_str:
                try:
                    # Adjust the date format to match "04/04/2018".
                    date_obj = datetime.strptime(date_str, "%m/%d/%Y").date()
                except ValueError:
                    date_obj = None

            abstract = row.get("Abstract", "").strip()
            journal = row.get("Journal", "").strip()
            try:
                citation_count = int(row.get("Citation Count", "").strip())
            except (ValueError, AttributeError):
                citation_count = None
            try:
                citation_velocity = float(row.get("Citation Velocity", "").strip())
            except (ValueError, AttributeError):
                citation_velocity = None
            doi = row.get("DOI", "").strip()
            link = row.get("Link", "").strip()
            external_ids = row.get("External IDs", "").strip()
            is_open_access = row.get("Is Open Access", "").strip().lower() in ["true", "1", "yes"]
            open_access_link = row.get("Open Access Link", "").strip()
            semantic_scholar_id = row.get("Semantic Scholar ID", "").strip()

            # Generate the citation key using the generate_cite_key function.
            cite_key = generate_cite_key(authors, year, title)

            # Create the Source instance.
            source = Source(
                cite_key=cite_key,
                title=title,
                year=year,
                date=date_obj,
                authors=authors,
                abstract=abstract,
                journal=journal,
                citation_count=citation_count,
                citation_velocity=citation_velocity,
                doi=doi,
                link=link,
                external_ids=external_ids,
                is_open_access=is_open_access,
                open_access_link=open_access_link,
                semantic_scholar_id=semantic_scholar_id
            )
            sources_to_add.append(source)

            # Get the rating from the CSV ("Topic Match Score").
            try:
                topic_match_score = float(row.get("Topic Match Score", "").strip())
            except (ValueError, AttributeError):
                topic_match_score = 0.0

            # Create the UnmindSourceRelevanceRating instance.
            rating = UnmindSourceRelevanceRating(
                cite_key=cite_key,
                rating=topic_match_score,
                topic=TOPIC_NAME
            )
            ratings_to_add.append(rating)

    # Bulk insert the sources and ratings.
    session.bulk_save_objects(sources_to_add)
    session.bulk_save_objects(ratings_to_add)
    session.commit()
    print(f"Imported {len(sources_to_add)} sources and {len(ratings_to_add)} ratings successfully.")

if __name__ == "__main__":
    unmind_import()


# Helper cell to clear Source table of all data

In [ ]:
import sys
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Add the root and models directories to the Python path.
sys.path.append(str(Path('.').resolve()))
sys.path.append(str((Path('.') / 'models').resolve()))

from config import NotebookConfig
from db_models import Source  # assuming your Source model is defined in models/db_models.py

# Create the engine and session.
engine = create_engine(f"sqlite:///{NotebookConfig.DB_FILE}")
Session = sessionmaker(bind=engine)
session = Session()

# Delete all rows from the Source table.
deleted_rows = session.query(Source).delete()
session.commit()

print(f"Deleted {deleted_rows} rows from the Source table.")

In [ ]:
import sys
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Add the root and models directories to the Python path.
sys.path.append(str(Path('.').resolve()))
sys.path.append(str((Path('.') / 'models').resolve()))

from config import NotebookConfig
from db_models import Source

# Create the database engine and session.
engine = create_engine(f"sqlite:///{NotebookConfig.DB_FILE}")
Session = sessionmaker(bind=engine)
session = Session()

# Query the database for all cite keys.
sources = session.query(Source.cite_key).all()

# Extract the cite keys from the query result.
cite_keys = [key for (key,) in sources]

# Print out the list of cite keys.
print("Cite Keys:")
for key in cite_keys:
    print(key)


In [ ]:
# Add new Sources even where I don't have full details

In [ ]:
import json
from sqlalchemy.orm import sessionmaker
from models.db_models import Source, engine
from config import NotebookConfig

# Create a new SQLAlchemy session.
Session = sessionmaker(bind=engine)
session = Session()

# Define the path to the JSON file.
json_file_path = NotebookConfig.INPUT_DIR / "new_papers.txt"

# Load the JSON content from the file.
with open(json_file_path, 'r') as file:
    papers_data = json.load(file)

# Iterate over each paper in the JSON.
for paper in papers_data:
    # Check if a source with the same cite_key already exists.
    existing_source = session.query(Source).filter_by(cite_key=paper["cite_key"]).first()
    if existing_source:
        print(f"Source with cite_key '{paper['cite_key']}' already exists. Skipping insertion.")
    else:
        # Create a new Source object using the data from the JSON.
        new_source = Source(
            cite_key=paper.get("cite_key"),
            title=paper.get("title"),
            authors=paper.get("authors"),
            year=paper.get("year")
        )
        session.add(new_source)
        print(f"Added new source: {paper['cite_key']}")

# Commit the session to save all changes to the database.
session.commit()


# Add Research Notes to the db

The json schema for notes this uses is the following (note that the chicago_citation is not used but is useful to have a visual check for the user.)

[
  {
    "cite_key": "anderson_2012_mtco",
    "chicago_citation": "Anderson, Ross J., et al. 2012. \"Measuring the Cost of Cybercrime.\"",
    "reports": [
      {
        "deep_research_report": "2025_04_03_undermind_cybercrime_and_intervention_metrics",
        "research_note": "This paper is referenced as part of the early foundational work (2012-2015) in cybercrime metrics, specifically pioneering systematic studies on the economic costs of cybercrime. It established frameworks to measure financial impacts and highlighted gaps in reliable data for policymaking."
      }
    ]
  },

In [ ]:
import json
from sqlalchemy.orm import sessionmaker
from models.db_models import Source, DeepResearchReport, ResearchNote, engine, NoteRouteType
from config import NotebookConfig

# -----------------------------
# CONFIGURATION: Set the route for the research notes.
# Options: "deep_research_report", "note_taking", "ai_notes"
ROUTE = "deep_research_report"  # Change this value as needed.
# -----------------------------

# Convert the route to the appropriate enum value.
try:
    note_route = NoteRouteType(ROUTE)
except ValueError:
    raise ValueError("Invalid route. Valid options are: deep_research_report, note_taking, ai_notes.")

# Create a new SQLAlchemy session.
Session = sessionmaker(bind=engine)
session = Session()

# Define the path to the JSON file.
json_file_path = NotebookConfig.INPUT_DIR / "notes_on_sources_1.txt"

# Load the JSON content from the file.
with open(json_file_path, 'r') as file:
    notes_data = json.load(file)

# Process each note entry in the JSON.
for note_entry in notes_data:
    cite_key = note_entry.get("cite_key")
    # Ensure the cite_key exists in the Source table.
    source = session.query(Source).filter_by(cite_key=cite_key).first()
    if not source:
        print(f"Source with cite_key '{cite_key}' not found. Skipping its notes.")
        continue
    
    # Process each report under the note entry.
    for report in note_entry.get("reports", []):
        report_name = report.get("deep_research_report")
        research_note_text = report.get("research_note")
        
        if not report_name or not research_note_text:
            print(f"Missing deep_research_report or research_note for cite_key '{cite_key}'. Skipping this report.")
            continue
        
        # Check if the deep research report already exists.
        deep_report = session.query(DeepResearchReport).filter_by(name=report_name).first()
        if not deep_report:
            deep_report = DeepResearchReport(name=report_name)
            session.add(deep_report)
            session.commit()  # Commit early so deep_report gets an ID.
            print(f"Added new DeepResearchReport: {report_name}")
        
        # Check if the note already exists based on cite_key and note_text.
        existing_note = session.query(ResearchNote).filter_by(cite_key=cite_key, note_text=research_note_text).first()
        if existing_note:
            print(f"Note for cite_key '{cite_key}' with the same text already exists. Skipping this note.")
            continue

        # Create a new ResearchNote for this report and set the cite_key.
        new_note = ResearchNote(
            note_text=research_note_text,
            route=note_route,
            deep_research_report=deep_report,
            cite_key=cite_key  # Associate note with the source's cite_key.
        )
        session.add(new_note)
        print(f"Added note for cite_key '{cite_key}' under report '{report_name}'.")

# Commit all changes.
session.commit()
print("Finished processing notes.")


In [ ]:
import re
import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import homogenize_latex_encoding
from sqlalchemy.orm import sessionmaker
from models.db_models import Source, engine
from config import NotebookConfig

def remove_extra_braces(s):
    """Remove all curly braces from a string."""
    return s.replace("{", "").replace("}", "")

def convert_bib_cite_key(bib_key, title):
    """
    Convert a BibTeX cite key to the custom format:
      - Lowercase the letters before the first digit.
      - Insert an underscore.
      - Keep only the first four digits.
      - Add an underscore and then the lowercase of the first letters of the first four words in the title.
    """
    # Remove extraneous braces from the title before processing.
    clean_title = remove_extra_braces(title)
    
    # Lowercase the letters before the first digit.
    prefix_match = re.match(r'^([A-Za-z]+)', bib_key)
    prefix = prefix_match.group(1).lower() if prefix_match else ''
    
    # Extract the first four digits.
    digits_match = re.search(r'(\d{4})', bib_key)
    digits = digits_match.group(1) if digits_match else ''
    
    # Get the first letters of the first four words of the cleaned title.
    words = re.findall(r'\w+', clean_title)
    letters = ''.join(word[0].lower() for word in words[:4])
    
    return f"{prefix}_{digits}_{letters}"

# Create a new SQLAlchemy session.
Session = sessionmaker(bind=engine)
session = Session()

# Define the path to your papers.bib file.
bib_file_path = NotebookConfig.INPUT_DIR / "papers.bib"

with open(bib_file_path, 'r') as bibtex_file:
    bibtex_str = bibtex_file.read()

# Parse the BibTeX file.
parser = BibTexParser(common_strings=True)
parser.customization = homogenize_latex_encoding
bib_database = bibtexparser.loads(bibtex_str, parser=parser)

for entry in bib_database.entries:
    original_key = entry.get('ID')  # the original BibTeX key
    raw_title = entry.get('title', '')
    # Remove extra braces from the title before saving.
    title = remove_extra_braces(raw_title)
    
    new_cite_key = convert_bib_cite_key(original_key, title)
    
    # Extract other fields from the BibTeX entry.
    url = entry.get('url')
    journal = entry.get('journal')
    booktitle = entry.get('booktitle')
    pages = entry.get('pages')
    volume = entry.get('volume')
    year = int(entry.get('year')) if entry.get('year') and entry.get('year').isdigit() else None
    authors = entry.get('author')
    
    # Check if a Source with the new cite key exists.
    source = session.query(Source).filter_by(cite_key=new_cite_key).first()
    
    if source:
        # Update the link field unconditionally.
        source.link = url
        # Update journal, volume, booktitle, pages if they are not already set (ignoring values like "unknown").
        if not source.journal and journal and journal.lower() != "unknown":
            source.journal = journal
        if not source.volume:
            source.volume = volume
        if not source.booktitle and booktitle and booktitle.lower() != "unknown":
            source.booktitle = booktitle
        if not source.pages:
            source.pages = pages
        print(f"Updated Source with cite_key {new_cite_key}")
    else:
        # Create a new Source with the provided fields.
        new_source = Source(
            cite_key=new_cite_key,
            title=title,
            authors=authors,
            year=year,
            link=url,
            journal=journal if journal and journal.lower() != "unknown" else None,
            booktitle=booktitle if booktitle and booktitle.lower() != "unknown" else None,
            pages=pages,
            volume=volume
        )
        session.add(new_source)
        print(f"Added new Source with cite_key {new_cite_key}")

# Commit the changes to the database.
session.commit()
print("Finished processing papers.bib")


# Helper Cell: Pretty Print a JSON and show in a table

In [ ]:
import json
import pandas as pd
from IPython.display import JSON, display

# Paste your JSON data as a Python list (or load it from a file).
data = [
{
"impact_id": 1,
"impact_text": "DETECTED - Cybercrime and transnational threats impacting the UK are DETECTED and notified to UK authorities.",
"indicators": [
{
"indicator_id": "1.1",
"indicator_text": "Volume, quality, and timeliness of actionable intelligence packages on UK-relevant threats/actors shared by partner country agencies with UK authorities (NCA, NCSC, Police).",
"potential_sources": [
"UK Law Enforcement/NCSC internal reporting systems.",
"Programme M&E records.",
"Reports from international bodies (Interpol, Europol)."
],
"relevant_insights": [
{
"insight_text": "Literature emphasizes international cooperation frameworks (e.g., Budapest Convention) as crucial structures enabling effective intelligence sharing (hussain_2023_acao, ugwu_2024_atet). Assessing adherence to or use of such frameworks can be a proxy for cooperation maturity.",
"cite_keys": [
"hussain_2023_acao",
"ugwu_2024_atet"
]
},
{
"insight_text": "While essential (shevko_2024_ccsa, anishchuk_2024_tpoc), the provided literature offers limited specific metrics for measuring the quality and actionability of shared intelligence itself; this often relies on recipient agency assessment.",
"cite_keys": [
"shevko_2024_ccsa",
"anishchuk_2024_tpoc"
]
}
],
"follow_up_source_reading_questions": [
"How did hussain_2023_acao or ugwu_2024_atet describe the mechanisms or impact of the Budapest Convention in practice?",
"Did shevko_2024_ccsa or anishchuk_2024_tpoc discuss specific mechanisms or best practices for effective international information sharing beyond stating its necessity?"
]
},
{
"indicator_id": "1.2",
"indicator_text": "Number of successful UK-relevant cybercrime detections (e.g., specific campaigns, actors, victim reports) originating from investigations or monitoring within partner countries enabled by programme support.",
"potential_sources": [
"Programme M&E records documenting specific detections linked to capabilities.",
"Case files from UK/Partner joint operations.",
"Partner country agency reports (if shared)."
],
"relevant_insights": [
{
"insight_text": "Researchers use technical measures like analysis of IDPS alert characteristics (miani_2015_apeo) or honeypot detection rates (rowe_2006_mteo) as indicators of detection activity within specific systems.",
"cite_keys": [
"miani_2015_apeo",
"rowe_2006_mteo"
]
},
{
"insight_text": "Studies suggest modernised legal frameworks are correlated with higher enforcement success/detection leading to prosecution (na_2024_elrt), indicating enabling legal environment is a key factor for this outcome.",
"cite_keys": [
"na_2024_elrt"
]
},
{
"insight_text": "Attributing detections specifically to programme support requires robust M&E tracking inputs (e.g., training, tools) to specific detection outputs/cases.",
"cite_keys": []
}
],
"follow_up_source_reading_questions": [
"What specific IDPS alert characteristics did miani_2015_apeo find useful as security incident indicators?",
"How did na_2024_elrt measure or evidence the 45% increase in prosecution rates attributed to modern laws?"
]
},
{
"indicator_id": "1.3",
"indicator_text": "Increased reporting by partner country citizens/businesses of UK-relevant cybercrime through established (potentially programme-supported) channels.",
"potential_sources": [
"Data from partner country reporting mechanisms (if accessible & filterable).",
"Programme M&E assessing usage of supported channels."
],
"relevant_insights": [
{
"insight_text": "Research demonstrates reporting interface effectiveness can be improved and measured through factors like usability, user trust, and perceived crime severity, based on user testing and surveys (bidgoli_2019_rnre, Undermind [17]).",
"cite_keys": [
"bidgoli_2019_rnre"
]
},
{
"insight_text": "Although victim surveys (bergh_2018_voci, breen_2022_alsm) provide prevalence estimates, the literature highlights underreporting and standardization issues (chen_2023_etgg, woods_2022_reoc) limiting their use for direct comparison or tracking programme impact on reporting behaviour itself, unless the programme directly improves survey mechanisms.",
"cite_keys": [
"bergh_2018_voci",
"breen_2022_alsm",
"chen_2023_etgg",
"woods_2022_reoc"
]
}
],
"follow_up_source_reading_questions": [
"What specific design elements or usability metrics did bidgoli_2019_rnre test or find effective for their reporting interface?",
"Did bergh_2018_voci discuss specific reasons for underreporting identified through their review of European victim surveys?"
]
},
{
"indicator_id": "1.4",
"indicator_text": "Increased identification and reporting by partner countries of malicious infrastructure (e.g., C2 servers, phishing sites) hosted within their jurisdiction that targets UK victims/interests.",
"potential_sources": [
"Partner country CERT/ISP/Law Enforcement reports (if shared).",
"NCSC/NCA data correlating infrastructure locations.",
"Threat intelligence feeds (public/commercial).",
"Analysis of honeypot/network traffic data."
],
"relevant_insights": [
{
"insight_text": "Literature demonstrates successful identification using technical sources like honeypot systems (thomas_2017_1dou, rowe_2006_mteo), analysis of backscatter/network traffic (wang_2020_dcpi, meland_2021_asms), and longitudinal analysis of DDoS/attack data (collier_2021_iiar).",
"cite_keys": [
"thomas_2017_1dou",
"rowe_2006_mteo",
"wang_2020_dcpi",
"meland_2021_asms",
"collier_2021_iiar"
]
},
{
"insight_text": "Cross-national datasets classifying malicious URLs/domains hosted globally can be used to track infrastructure prevalence in partner countries (amin_2020_tsao).",
"cite_keys": [
"amin_2020_tsao"
]
}
],
"follow_up_source_reading_questions": [
"What type of honeypots or data (e.g., UDP reflectors) did thomas_2017_1dou find effective for tracking DDoS infrastructure?",
"What methodology did amin_2020_tsao use for classifying URLs and identifying high-risk countries in their dataset?"
]
}
]
},
{
"impact_id": 2,
"impact_text": "DISRUPTED - Cybercrime and transnational threats impacting UK interests are DISRUPTED.",
"indicators": [
{
"indicator_id": "2.1",
"indicator_text": "Number and scale of cybercriminal operations (actors, infrastructure, markets) impacting the UK that are disrupted or dismantled by partner country authorities (potentially jointly or with UK support/intel derived from capacity building).",
"potential_sources": [
"Programme M&E records tracking disruptions.",
"UK Law Enforcement/NCSC reporting.",
"Partner agency reports (if shared).",
"Press releases/official reports."
],
"relevant_insights": [
{
"insight_text": "Research uses longitudinal analysis of attack data (e.g., time series) to measure the impact of disruptions like infrastructure takedowns (collier_2021_iiar).",
"cite_keys": [
"collier_2021_iiar"
]
},
{
"insight_text": "Successful disruption relies heavily on international cooperation structures and capable legal frameworks enabling swift action (shevko_2024_ccsa, hussain_2023_acao).",
"cite_keys": [
"shevko_2024_ccsa",
"hussain_2023_acao"
]
}
],
"follow_up_source_reading_questions": [
"What specific metrics (e.g., attack volume, type, source) did collier_2021_iiar track in their time-series analysis before/after disruption?",
"Did shevko_2024_ccsa provide examples of specific cooperative actions that led to successful disruptions?"
]
},
{
"indicator_id": "2.2",
"indicator_text": "Number of arrests and successful prosecutions in partner countries of cybercriminals targeting UK victims/interests, resulting from programme-supported capabilities or investigations.",
"potential_sources": [
"Programme M&E records linking support to cases.",
"Partner judicial/police records (if accessible).",
"UK Law Enforcement liaison reporting.",
"MLAT request outcomes."
],
"relevant_insights": [
{
"insight_text": "Studies indicate a correlation between modernised cybercrime laws/legal frameworks and increased prosecution rates (na_2024_elrt), making legal capacity a key enabler.",
"cite_keys": [
"na_2024_elrt"
]
},
{
"insight_text": "Measuring the programme's specific contribution requires M&E to document how supported capabilities (e.g., digital forensics, legal training) were utilised in successful cases.",
"cite_keys": []
}
],
"follow_up_source_reading_questions": [
"What was the methodology used by na_2024_elrt to establish the correlation between modern laws and the 45% higher prosecution rate?",
"Are there specific types of legal provisions highlighted in hussain_2023_acao or na_2024_elrt that are particularly crucial for enabling prosecutions?"
]
},
{
"indicator_id": "2.3",
"indicator_text": "Measurable reduction in specific UK-impacting malicious activities (e.g., spam from partner country IPs, specific malware C&C traffic, successful phishing attempts attributed to actors in partner country) following partner country disruption actions.",
"potential_sources": [
"NCSC Active Cyber Defence reports/data.",
"Threat intelligence feeds.",
"Network traffic analysis (pre/post disruption).",
"Cybersecurity vendor data."
],
"relevant_insights": [
{
"insight_text": "Researchers successfully use analysis of technical data sources (network traces, logs, honeypots, backscatter) pre- and post-intervention to quantify reductions in malicious activity (meland_2021_asms, wang_2020_dcpi, collier_2021_iiar).",
"cite_keys": [
"meland_2021_asms",
"wang_2020_dcpi",
"collier_2021_iiar"
]
},
{
"insight_text": "While reduction is measurable, attributing it solely to disruption requires careful analysis, as threat actors adapt tactics (collier_2021_iiar implies this with changing infrastructure). Long-term monitoring is needed.",
"cite_keys": [
"collier_2021_iiar"
]
}
],
"follow_up_source_reading_questions": [
"What specific data sources (e.g., network vs system data, specific threat feeds) did meland_2021_asms find most useful for measuring activity changes?",
"Did collier_2021_iiar observe specific ways actors adapted following disruption efforts?"
]
},
{
"indicator_id": "2.4",
"indicator_text": "Adoption and implementation by partner countries of specific policies, regulations, or technical standards (promoted through capacity building) aimed at disrupting cybercrime capabilities (e.g., ISP actions against botnets, cryptocurrency regulations).",
"potential_sources": [
"Programme M&E tracking policy adoption.",
"Partner government publications/legislation.",
"International bodies monitoring policy implementation."
],
"relevant_insights": [
{
"insight_text": "The literature highlights specific legislation (e.g., Pakistan's Prevention of Electronic Act - hussain_2023_acao) and governance frameworks (ugwu_2024_atet) as key countermeasures; tracking adoption of similar measures is a valid indicator.",
"cite_keys": [
"hussain_2023_acao",
"ugwu_2024_atet"
]
},
{
"insight_text": "Policy surveillance methodology, as proposed by dupont_2019_eteo, provides a structured approach for tracking the adoption and features of such policies globally or regionally.",
"cite_keys": [
"dupont_2019_eteo"
]
}
],
"follow_up_source_reading_questions": [
"Did dupont_2019_eteo suggest specific data points or features to track in a cybercrime policy monitoring framework?",
"What specific aspects of Pakistan's Act did hussain_2023_acao highlight as being crucial for combating cybercrime?"
]
}
]
},
{
"impact_id": 3,
"impact_text": "DETERRED - Cybercriminals and states are DETERRED by higher risks, higher costs or reduced return.",
"indicators": [
{
"indicator_id": "3.1",
"indicator_text": "Evidence of increased operational cost or difficulty for specific UK-targeting threat actors operating from or transiting through partner countries, attributed to enhanced partner capabilities (e.g., forcing changes in TTPs, infrastructure relocation).",
"potential_sources": [
"UK/Partner threat intelligence analysis (qualitative).",
"Academic studies on cybercrime economics/actor adaptation.",
"Long-term analysis of disruption impacts (Indicator 2.3)."
],
"relevant_insights": [
{
"insight_text": "Researchers identify 'attacker cost' (often measured by time or simulated effort) as a key metric for evaluating countermeasure effectiveness, directly relating to deterrence (sandoval_2010_miac, Elicit report).",
"cite_keys": [
"sandoval_2010_miac"
]
},
{
"insight_text": "Studies suggest countermeasures are most effective (and likely deterring) when they target the financial 'revenue streams' or business models of attackers (fajar_2020_tist, an_2018_adat). Observing adaptation away from disrupted revenue models indicates success.",
"cite_keys": [
"fajar_2020_tist",
"an_2018_adat"
]
},
{
"insight_text": "Measuring increased cost often relies on qualitative intelligence observing changes in actor behaviour/TTPs, rather than direct financial data from criminals.",
"cite_keys": []
}
],
"follow_up_source_reading_questions": [
"How did sandoval_2010_miac operationalize or simulate 'attacker cost' in their study?",
"What methods did fajar_2020_tist or an_2018_adat use to analyze the economic impact on attacker revenue streams?"
]
},
{
"indicator_id": "3.2",
"indicator_text": "Increased number of partner country-led actions (e.g., policy statements, joint attributions, diplomatic actions, operational disruptions) against state or state-sponsored actors whose activities impact the UK.",
"potential_sources": [
"Programme M&E tracking partner actions.",
"Public statements from partner governments.",
"Reporting on international diplomatic engagements.",
"Joint attribution statements."
],
"relevant_insights": [
{
"insight_text": "While direct measurement of state actor deterrence is absent in the provided literature, tracking coordinated international actions (advocated by shevko_2024_ccsa, ugwu_2024_atet, anishchuk_2024_tpoc) serves as an output indicator for collective signalling intended to deter.",
"cite_keys": [
"shevko_2024_ccsa",
"ugwu_2024_atet",
"anishchuk_2024_tpoc"
]
},
{
"insight_text": "State & Transnational Threats are part of the Theory of Change (Node 17, 18, 19); measuring partner actions in this space is feasible, measuring the deterrent effect is the challenge.",
"cite_keys": []
}
],
"follow_up_source_reading_questions": [
"Did the cooperation papers (shevko_2024_ccsa, ugwu_2024_atet, anishchuk_2024_tpoc) give examples of specific types of successful joint actions taken against state-level threats?",
]
},
{
"indicator_id": "3.3",
"indicator_text": "Demonstrable shift by partner countries towards proactive cybercrime prevention and improved cybersecurity posture (e.g., adoption of frameworks, public awareness campaigns, CERT maturity) resulting from capacity building.",
"potential_sources": [
"Programme M&E tracking capabilities/framework adoption.",
"National cybersecurity assessments/indices.",
"Partner national cybersecurity strategy documents/reports.",
"Regional body reports."
],
"relevant_insights": [
{
"insight_text": "Researchers identify the implementation of cybersecurity frameworks and protocols as an effective countermeasure (cassidy_2024_etac, saeed_2021_catp) which contributes to a stronger defensive posture, increasing baseline difficulty for attackers.",
"cite_keys": [
"cassidy_2024_etac",
"saeed_2021_catp"
]
},
{
"insight_text": "Frameworks like the GQIM model (albluwi_2017_fpec) or maturity models (lippmann_2012_csmp) offer structured ways, used in research/practice, to assess and improve security posture over time.",
"cite_keys": [
"albluwi_2017_fpec",
"lippmann_2012_csmp"
]
},
{
"insight_text": "Country-level cybersecurity indicators, such as those used by yarovenko_2020_sfdr, can track national progress in areas like policy development and education, reflecting posture improvements.",
"cite_keys": [
"yarovenko_2020_sfdr"
]
}
],
"follow_up_source_reading_questions": [
"What specific frameworks did cassidy_2024_etac or saeed_2021_catp evaluate or find effective?",
"What are the key components or levels of the metric maturity model proposed by lippmann_2012_csmp?",
"What were the 12 specific country-level indicators used by yarovenko_2020_sfdr?"
]
},
{
"indicator_id": "3.4",
"indicator_text": "Reduction in the hosting or transit of UK-relevant cybercriminal infrastructure/services within partner country jurisdictions over time.",
"potential_sources": [
"Threat intelligence mapping (commercial/public).",
"NCSC/NCA data analysis.",
"Long-term analysis from technical monitoring (Indicator 1.4)."
],
"relevant_insights": [
{
"insight_text": "Mapping the global geography of cybercrime using methods like expert surveys (bruce_2024_mtgg) or spatial analysis of technical data (amin_2020_tsao) allows tracking of infrastructure concentration and potential shifts away from 'hotspot' partner countries.",
"cite_keys": [
"bruce_2024_mtgg",
"amin_2020_tsao"
]
},
{
"insight_text": "A sustained decrease in technical indicators associated with malicious hosting (e.g., number of phishing URLs, malware domains attributed to partner country IPs) provides evidence for reduced criminal activity/hosting.",
"cite_keys": [
"chen_2023_etgg"
]
}
],
"follow_up_source_reading_questions": [
"What were the five major cybercrime categories used in the expert survey by bruce_2024_mtgg for the World Cybercrime Index?",
"What specific spatial analysis techniques (e.g., SaTScan parameters) did amin_2020_tsao use to identify clusters of malicious URLs?"
]
}
]
}
]

# Option 1: Use IPython's interactive JSON viewer.
display(JSON(data))

# Option 2: Flatten the JSON into a table for a consolidated view.
rows = []
for impact in data:
    for indicator in impact.get("indicators", []):
        potential_sources = "\n".join(indicator.get("potential_sources", []))
        insights = "\n\n".join(
            f"Insight: {insight.get('insight_text','')}\nCite Keys: {', '.join(insight.get('cite_keys', []))}"
            for insight in indicator.get("relevant_insights", [])
        )
        rows.append({
            "Impact ID": impact.get("impact_id"),
            "Impact Text": impact.get("impact_text"),
            "Indicator ID": indicator.get("indicator_id"),
            "Indicator Text": indicator.get("indicator_text"),
            "Potential Sources": potential_sources,
            "Relevant Insights": insights
        })

df = pd.DataFrame(rows)
display(df)


# Add follow up questions of sources to db

In [6]:
import json
import re
from sqlalchemy.orm import sessionmaker
from models.db_models import FollowUpQuestion, SourcesNeededForFollowUps, engine
from config import NotebookConfig

# Create a new SQLAlchemy session.
Session = sessionmaker(bind=engine)
session = Session()

# Define a regex pattern to extract cite_keys (pattern: letters_digits_letters)
# This pattern matches one or more lowercase letters, an underscore, exactly 4 digits, another underscore,
# and one or more lowercase letters.
pattern = r'([a-z]+_[0-9]{4}_[a-z]+)'

# Define the path to the proposed_indicators.txt file.
json_file_path = NotebookConfig.INPUT_DIR / "proposed_indicators.txt"

# Load the JSON content.
with open(json_file_path, 'r') as file:
    toc_data = json.load(file)

# Loop through each ToC node and its indicators.
for toc in toc_data:
    toc_id = toc.get("toc_id")
    for indicator in toc.get("indicators", []):
        indicator_id = indicator.get("indicator_id")
        questions = indicator.get("follow_up_source_reading_questions", [])
        for question_text in questions:
            # Create a new FollowUpQuestion record.
            new_question = FollowUpQuestion(
                toc_id=toc_id,
                indicator_id=indicator_id,
                question_text=question_text
            )
            session.add(new_question)
            # Flush the session to assign an ID to new_question.
            session.flush()
            
            # Extract cite_keys from the question text.
            matches = re.findall(pattern, question_text)
            for cite_key in matches:
                new_source_needed = SourcesNeededForFollowUps(
                    question_id=new_question.id,
                    cite_key=cite_key
                )
                session.add(new_source_needed)
                print(f"Added source needed for question id {new_question.id}: {cite_key}")
            
            print(f"Added question for toc_id {toc_id}, indicator {indicator_id}: {question_text}")

# Commit all changes.
session.commit()
print("Finished extracting and saving follow-up questions and associated source cite_keys.")


Added source needed for question id 1: bergh_2018_voci
Added source needed for question id 1: woods_2022_reoc
Added question for toc_id 1, indicator I1.1: What specific survey questions or methodologies did `bergh_2018_voci` and `woods_2022_reoc` find most common or problematic for measuring cybercrime detection/victimisation?
Added source needed for question id 2: breen_2022_alsm
Added question for toc_id 1, indicator I1.1: How did `breen_2022_alsm` define and implement 'social-reporting techniques' in their survey to improve cybercrime measurement?
Added source needed for question id 3: collier_2021_iiar
Added question for toc_id 1, indicator I1.1: What specific configuration details (e.g., protocols, services mimicked) did `collier_2021_iiar` or `thomas_2017_1dou` use for their honeypots to measure DDoS attacks?
Added source needed for question id 4: meland_2021_asms
Added source needed for question id 4: miani_2015_apeo
Added question for toc_id 1, indicator I1.1: What specific dat

In [1]:
from sqlalchemy import create_engine, text
from config import NotebookConfig

# Create the database engine.
engine = create_engine(f"sqlite:///{NotebookConfig.DB_FILE}")

# SQL statement to create a view with unique cite_keys, along with link and open_access_link.
create_view_sql = text("""
    CREATE VIEW IF NOT EXISTS vw_unique_cite_keys_for_follow_ups AS
    SELECT DISTINCT s.cite_key, s.link, s.open_access_link
    FROM sources_needed_for_follow_ups snf
    JOIN source s ON snf.cite_key = s.cite_key;
""")

# Execute the SQL statement.
with engine.begin() as connection:
    connection.execute(create_view_sql)

print("View 'vw_unique_cite_keys_for_follow_ups' with link and open_access_link has been created (if it did not already exist).")



View 'vw_unique_cite_keys_for_follow_ups' with link and open_access_link has been created (if it did not already exist).


# Helper cell: print the indicators from the potential_indicators.txt file

In [3]:
import json
import pandas as pd
from config import NotebookConfig

# Define the path to the JSON file.
json_file_path = NotebookConfig.INPUT_DIR / "proposed_indicators_and_sources.txt"

# Load the JSON content.
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Prepare a list to hold rows for our table.
rows = []

# Iterate over each topic in the JSON data.
for toc in data:
    topic_id = toc.get("toc_id")
    topic_text = toc.get("toc_text")
    topic_level = toc.get("toc_level")
    
    # Process each indicator in the topic.
    for indicator in toc.get("indicators", []):
        indicator_id = indicator.get("indicator_id")
        indicator_text = indicator.get("indicator_text")
        potential_sources = indicator.get("potential_sources", [])
        
        # Ensure potential_sources is a list.
        if not isinstance(potential_sources, list):
            potential_sources = [potential_sources]
        # If there are no sources, use an empty string.
        if not potential_sources:
            potential_sources = ['']
        
        # For each potential source, create a row.
        # The first source gets the full topic/indicator details; subsequent ones get blank details.
        for i, source in enumerate(potential_sources):
            if i == 0:
                rows.append({
                    "topic_id": topic_id,
                    "topic_text": topic_text,
                    "topic_level": topic_level,
                    "indicator_id": indicator_id,
                    "indicator_text": indicator_text,
                    "potential_sources": source
                })
            else:
                rows.append({
                    "topic_id": "",
                    "topic_text": "",
                    "topic_level": "",
                    "indicator_id": "",
                    "indicator_text": "",
                    "potential_sources": source
                })

# Create a DataFrame from the rows.
df = pd.DataFrame(rows)

# Reorder columns if desired.
df = df[["topic_id", "topic_text", "topic_level", "indicator_id", "indicator_text", "potential_sources"]]

# Save the DataFrame to a CSV file in the output directory with UTF-8 encoding.
output_path = NotebookConfig.OUTPUT_DIR / "proposed_indicator_table_clean.csv"
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Table saved to {output_path}")



Table saved to output_docs/proposed_indicator_table_clean.csv
